In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import folium
import geopandas as gpd

In [2]:
confirmed = pd.read_csv('./data/jhu_data/time_series_19-covid-Confirmed.csv')

In [3]:
confirmed.head()

Province/State Country/Region      Lat      Long  1/22/20  1/23/20  1/24/20  \
0            NaN       Thailand  15.0000  101.0000        2        3        5   
1            NaN          Japan  36.0000  138.0000        2        1        2   
2            NaN      Singapore   1.2833  103.8333        0        1        3   
3            NaN          Nepal  28.1667   84.2500        0        0        0   
4            NaN       Malaysia   2.5000  112.5000        0        0        0   

   1/25/20  1/26/20  1/27/20   ...     3/4/20  3/5/20  3/6/20  3/7/20  3/8/20  \
0        7        8        8   ...         43      47      48      50      50   
1        2        4        4   ...        331     360     420     461     502   
2        3        4        5   ...        110     117     130     138     150   
3        1        1        1   ...          1       1       1       1       1   
4        3        4        4   ...         50      50      83      93      99   

   3/9/20  3/10/20  3/11/20  3/12/20  3/13/20  
0      50       53       59       70       75  
1     511      581      639      639      701  
2     150      160      178      178      200  
3       1        1        1        1        1  
4     117      129      149      149      197  

[5 rows x 56 columns]

In [4]:
dates = confirmed.columns[4:]
ndays = len(dates)

In [5]:
confirmed_melted = confirmed.melt(id_vars=['Province/State', 
                                         'Country/Region', 
                                         'Lat', 
                                         'Long'], 
                                value_vars=dates, 
                                var_name='Date', 
                                value_name='Confirmed')

In [33]:
len(confirmed_melted)

21892

In [7]:
first_day = ≈['Date'].iloc[0]
last_day = confirmed_melted['Date'].iloc[-1]

In [36]:
mask1 = confirmed_melted['Date'].str.contains(first_day)
first_day_df = confirmed_melted[mask1]
first_day_df.head()

Province/State Country/Region      Lat      Long     Date  Confirmed
0            NaN       Thailand  15.0000  101.0000  1/22/20          2
1            NaN          Japan  36.0000  138.0000  1/22/20          2
2            NaN      Singapore   1.2833  103.8333  1/22/20          0
3            NaN          Nepal  28.1667   84.2500  1/22/20          0
4            NaN       Malaysia   2.5000  112.5000  1/22/20          0

In [69]:
data_to_plot = first_day_df[['Country/Region', 'Confirmed']]
data_to_plot.rename(columns={'Country/Region':'State'},inplace=True)
data_to_plot = data_to_plot.groupby('State').sum().reset_index()
data_to_plot.head()

State  Confirmed
0          Afghanistan          0
1              Albania          0
2              Algeria          0
3              Andorra          0
4  Antigua and Barbuda          0

In [39]:
hist_indicator = 'Confirmed'

In [84]:
countries_geo = 'world-countries.json'
state_data = data_to_plot

m = folium.Map(location=[50, 0], zoom_start=3)

bins = [0,100,200,300,400,500,600]

folium.Choropleth(
    geo_data=countries_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Confirmed'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins,
    legend_name='Confirmed Cases'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [86]:
mask1 = confirmed_melted['Date'].str.contains(last_day)
last_day = confirmed_melted[mask1]
last_day_df.head()

Province/State Country/Region      Lat      Long     Date  Confirmed
21471            NaN       Thailand  15.0000  101.0000  3/13/20         75
21472            NaN          Japan  36.0000  138.0000  3/13/20        701
21473            NaN      Singapore   1.2833  103.8333  3/13/20        200
21474            NaN          Nepal  28.1667   84.2500  3/13/20          1
21475            NaN       Malaysia   2.5000  112.5000  3/13/20        197

In [87]:
data_to_plot = last_day_df[['Country/Region', 'Confirmed']]
data_to_plot.rename(columns={'Country/Region':'State'},inplace=True)
data_to_plot = data_to_plot.groupby('State').sum().reset_index()
data_to_plot.head()

State  Confirmed
0          Afghanistan          7
1              Albania         33
2              Algeria         26
3              Andorra          1
4  Antigua and Barbuda          1

In [94]:
countries_geo = 'world-countries.json'
state_data = data_to_plot

m = folium.Map(location=[50, 0], zoom_start=3)

bins = list(data_to_plot['Confirmed'].quantile([0, 0.25, 0.5, 0.75, 1]))
print(bins)

folium.Choropleth(
    geo_data=countries_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Confirmed'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins,
    legend_name='Confirmed Cases'
).add_to(m)

folium.LayerControl().add_to(m)

[1.0, 5.0, 26.0, 112.0, 80945.0]


In [95]:
m.save('last_day.html')
# Import the Folium interactive html file


from IPython.display import HTML
HTML('<iframe src=last_day.html width=700 height=450></iframe>')